# Versión dos del modelo
En "maria_modelv1.ipynb" he creado un documento sqlite con el dataframe de de los datos de la clasificación de cada equipo (ejercicio 10). Voy a intentar crear el modelo de la versión v1 utilizando los dos data frames (laliga.sqlite, clasification.sqlite) sin hacerles un merge porque me falla la RAM.
 
Update1: Tengo que hacerles un merge a los dataframes porque tienen que tener las mismas dimesiones.

Update2: LA kernel muere al juntar los dos dataframes. (En la versión 1, para una season iba ok, pero es demasiado poco).

Update3: Ahora funciona, pero el el dataframe de train solo tiene una season.

In [1]:
# Importo los paquetes
import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier # el modelo
from sklearn.metrics import accuracy_score, classification_report # para ver cómo funciona
from sklearn.preprocessing import LabelEncoder # poner los datos tipo int

In [2]:
# Importo los dataframes
with sqlite3.connect("../laliga.sqlite") as conn:
        df = pd.read_sql("SELECT * FROM Matches", conn)
conn.close() 

In [3]:
# Importo los dataframes
with sqlite3.connect("clasification.sqlite") as conn:
        df_teams = pd.read_sql("SELECT * FROM clasification", conn)
conn.close()  

In [4]:
# Si pongo directamente df_games =con(...) me da error en la siguiente celda
df_games=df  

In [5]:
# Le creo a df_games la columna de result (home, visitor, tie) que va a ser mi variable Y en el modelo
df_games["result"] = None
#estas dos columnas nos harían falta
df_games["goals_home"] = df_games["score"].str.split(":").str[0]
df_games["goals_visitor"] = df_games["score"].str.split(":").str[1]

df_games.loc[df["goals_home"] > df_games["goals_visitor"], "result"] = 'home'
df_games.loc[df["goals_home"] == df_games["goals_visitor"], "result"] = 'tie'
df_games.loc[df["goals_home"] < df_games["goals_visitor"], "result"] = 'visitor'

In [6]:
df_games.drop(["goals_home", "goals_visitor", "time", "date", "score"], axis=1, inplace=True)  
df_games

,season,division,matchday,home_team,away_team,result
0,1928-1929,1,1,Arenas Club,Athletic Madrid,visitor
1,1928-1929,1,1,Espanyol,Real Unión,home
2,1928-1929,1,1,Real Madrid,Catalunya,home
3,1928-1929,1,1,Donostia,Athletic,tie
4,1928-1929,1,1,Racing,Barcelona,visitor
...,...,...,...,...,...,...
48775,2021-2022,2,42,Real Oviedo,UD Ibiza,None
48776,2021-2022,2,42,Real Sociedad B,Real Zaragoza,None
48777,2021-2022,2,42,Sporting Gijón,UD Las Palmas,None
48778,2021-2022,2,42,CD Tenerife,FC Cartagena,None


# Empieza el modelo (RandomForestClassifier)
Nuestro modelo/problema es de tipo clasificación. La predicción nos tiene que dar uno de los tres valores, NO es una variable continua.

Ya tenemos los datos. Ahora hay que definir nuestras variables. X son los datos que vamos a tener en cuenta, e Y es la "variable etiqueta", es decir, la que tenemos que predecir. La etiqueta tiene que ser de tipo int, empezamos con eso.

In [7]:
division = 1
season = "2019-2020"
matchday = 3
season_train = "2000:2002"


In [8]:
# Luego, aplicar las operaciones de cadena de caracteres
df_teams_train = df_teams

# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres
df_teams_train["season"] = df_teams_train["season"].astype(str)

df_teams_train = df_teams_train[(df_teams_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_teams_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]
df_teams_train


,season,division,rank,team,matchday,GF,GA,GD,W,L,T,Pts
33680,2001-2002,1,1.0,Dep. La Coruña,1,4.0,0.0,4.0,1.0,0.0,0.0,3.0
33681,2001-2002,1,2.0,Celta de Vigo,1,3.0,0.0,3.0,1.0,0.0,0.0,3.0
33682,2001-2002,1,3.0,UD Las Palmas,1,3.0,0.0,3.0,1.0,0.0,0.0,3.0
33683,2001-2002,1,4.0,Athletic,1,3.0,1.0,2.0,1.0,0.0,0.0,3.0
33684,2001-2002,1,5.0,Alavés,1,2.0,0.0,2.0,1.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
37043,2000-2001,2,18.0,Elche CF,42,45.0,56.0,-11.0,10.0,16.0,16.0,46.0
37044,2000-2001,2,19.0,SD Compostela,42,41.0,65.0,-24.0,11.0,19.0,12.0,45.0
37045,2000-2001,2,20.0,Uni. Las Palmas,42,34.0,63.0,-29.0,8.0,19.0,15.0,39.0
37046,2000-2001,2,21.0,Getafe,42,42.0,65.0,-23.0,8.0,23.0,11.0,35.0


In [9]:
df_games_train = df_games
# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres

df_games_train["season"] = df_games_train["season"].astype(str)

# Luego, aplicar las operaciones de cadena de caracteres
df_games_train = df_games_train[(df_games_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_games_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]
df_games_train


,season,division,matchday,home_team,away_team,result
17838,2000-2001,1,1,Real Sociedad,Racing,tie
17839,2000-2001,1,1,Real Zaragoza,Espanyol,visitor
17840,2000-2001,1,1,Barcelona,Málaga CF,home
17841,2000-2001,1,1,Dep. La Coruña,Athletic,home
17842,2000-2001,1,1,Real Madrid,Valencia,home
...,...,...,...,...,...,...
39155,2001-2002,2,42,Córdoba CF,Racing Ferrol,tie
39156,2001-2002,2,42,Real Murcia,Real Jaén CF,home
39157,2001-2002,2,42,Poli Ejido,Racing,home
39158,2001-2002,2,42,CD Numancia,Levante,visitor


In [10]:
# Hacemos el merge del dataframe para train
df_train = pd.merge(df_games_train, df_teams_train, left_on='home_team', right_on='team', how='left', suffixes=("_home", ""))
df_train = pd.merge(df_train, df_teams_train, left_on='away_team', right_on='team', how='left', suffixes=("_away",""))
df_train

,season_home,division_home,matchday_home,home_team,away_team,result,season_away,division_away,rank_away,team_away,...,rank,team,matchday,GF,GA,GD,W,L,T,Pts
0,2000-2001,1,1,Real Sociedad,Racing,tie,2001-2002,1,16.0,Real Sociedad,...,14.0,Racing,1,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,2000-2001,1,1,Real Sociedad,Racing,tie,2001-2002,1,16.0,Real Sociedad,...,14.0,Racing,2,1.0,1.0,0.0,0.0,0.0,2.0,2.0
2,2000-2001,1,1,Real Sociedad,Racing,tie,2001-2002,1,16.0,Real Sociedad,...,18.0,Racing,3,1.0,3.0,-2.0,0.0,1.0,2.0,2.0
3,2000-2001,1,1,Real Sociedad,Racing,tie,2001-2002,1,16.0,Real Sociedad,...,13.0,Racing,4,2.0,3.0,-1.0,1.0,1.0,2.0,5.0
4,2000-2001,1,1,Real Sociedad,Racing,tie,2001-2002,1,16.0,Real Sociedad,...,15.0,Racing,5,2.0,5.0,-3.0,1.0,2.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9759515,2001-2002,2,42,Gimnàstic,CF Extremadura,home,2001-2002,2,20.0,Gimnàstic,...,13.0,CF Extremadura,38,35.0,42.0,-7.0,14.0,16.0,8.0,50.0
9759516,2001-2002,2,42,Gimnàstic,CF Extremadura,home,2001-2002,2,20.0,Gimnàstic,...,12.0,CF Extremadura,39,37.0,42.0,-5.0,15.0,16.0,8.0,53.0
9759517,2001-2002,2,42,Gimnàstic,CF Extremadura,home,2001-2002,2,20.0,Gimnàstic,...,11.0,CF Extremadura,40,39.0,43.0,-4.0,16.0,16.0,8.0,56.0
9759518,2001-2002,2,42,Gimnàstic,CF Extremadura,home,2001-2002,2,20.0,Gimnàstic,...,11.0,CF Extremadura,41,40.0,44.0,-4.0,16.0,16.0,9.0,57.0


In [11]:
#Esto es otra opción para seleccionar las temporadas a las que vamos a hacer el train
#Define el rango de temporadas como una cadena
#temporadas = "2000:2010"

#Divide el rango en años de inicio y final
#temporada_inicio, temporada_fin = temporadas.split(":")m
#temporada_inicio = int(temporada_inicio)
#temporada_fin = int(temporada_fin)

#Filtra las filas correspondientes al rango de temporadas
#df_entrenamiento = df[(df['season'] >= f"{temporada_inicio}-{temporada_inicio + 1}")
#                      & (df['season'] <= f"{temporada_fin - 1}-{temporada_fin}")]

In [12]:
# Supongamos que tienes un DataFrame df con una columna 'etiqueta' de tipo 'object'
le = LabelEncoder()
df_train['prediction'] = le.fit_transform(df_train['result'])

In [13]:
# Selecciona las características (X_train) y etiquetas (y_train)
X_train = df_train[['rank', 'GD', "W", "Pts"]]  
y_train = df_train['prediction']  

In [14]:
df_teams_test = df_teams[df_teams["season"] == season]
df_games_test = df_games[df_games["season"] == season]

In [15]:
# Hacemos el merge del dataframe para hacer el test
df_test = pd.merge(df_games_test, df_teams_test, left_on='home_team', right_on='team', how='left', suffixes=("_home", ""))
df_test = pd.merge(df_test, df_teams_test, left_on='away_team', right_on='team', how='left', suffixes=("_away",""))
df_test

,season_home,division_home,matchday_home,home_team,away_team,result,season_away,division_away,rank_away,team_away,...,rank,team,matchday,GF,GA,GD,W,L,T,Pts
0,2019-2020,1,1,Athletic,Barcelona,home,2019-2020,1,6.0,Athletic,...,15.0,Barcelona,1,0.0,1.0,-1.0,0.0,1.0,0.0,0.0
1,2019-2020,1,1,Athletic,Barcelona,home,2019-2020,1,6.0,Athletic,...,9.0,Barcelona,2,5.0,3.0,2.0,1.0,1.0,0.0,3.0
2,2019-2020,1,1,Athletic,Barcelona,home,2019-2020,1,6.0,Athletic,...,8.0,Barcelona,3,7.0,5.0,2.0,1.0,1.0,1.0,4.0
3,2019-2020,1,1,Athletic,Barcelona,home,2019-2020,1,6.0,Athletic,...,5.0,Barcelona,4,12.0,7.0,5.0,2.0,1.0,1.0,7.0
4,2019-2020,1,1,Athletic,Barcelona,home,2019-2020,1,6.0,Athletic,...,8.0,Barcelona,5,12.0,9.0,3.0,2.0,2.0,1.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363683,2019-2020,2,42,Dep. La Coruña,CF Fuenlabrada,home,2019-2020,2,19.0,Dep. La Coruña,...,10.0,CF Fuenlabrada,38,41.0,37.0,4.0,12.0,11.0,15.0,51.0
1363684,2019-2020,2,42,Dep. La Coruña,CF Fuenlabrada,home,2019-2020,2,19.0,Dep. La Coruña,...,9.0,CF Fuenlabrada,39,42.0,37.0,5.0,13.0,11.0,15.0,54.0
1363685,2019-2020,2,42,Dep. La Coruña,CF Fuenlabrada,home,2019-2020,2,19.0,Dep. La Coruña,...,7.0,CF Fuenlabrada,40,43.0,37.0,6.0,14.0,11.0,15.0,57.0
1363686,2019-2020,2,42,Dep. La Coruña,CF Fuenlabrada,home,2019-2020,2,19.0,Dep. La Coruña,...,6.0,CF Fuenlabrada,41,46.0,38.0,8.0,15.0,11.0,15.0,60.0


In [16]:
le = LabelEncoder()
df_test['prediction'] = le.fit_transform(df_test['result'])

In [17]:
X_test = df_test[['rank', 'GD', "W", "Pts"]] # he cogido estás aleatoriamente
y_test = df_test['prediction']

In [18]:
# Crear y entrenar un modelo de clasificación (por ejemplo, Random Forest)
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [19]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [20]:
# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

Precisión del modelo: 0.4279849936349077


In [21]:
# Mostrar un informe de clasificación
print(classification_report(y_test, y_pred, target_names=['home_win(1)', 'visitor_win(2)', 'tie(X)']))

                precision    recall  f1-score   support

   home_win(1)       0.44      0.93      0.59    582888
visitor_win(2)       0.34      0.08      0.13    428568
        tie(X)       0.35      0.03      0.05    352232

      accuracy                           0.43   1363688
     macro avg       0.38      0.34      0.26   1363688
  weighted avg       0.38      0.43      0.31   1363688

